<a href="https://colab.research.google.com/github/oojjww1/Data/blob/main/CodeStates(Project_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') # 한글 지원되도록 설정.
mpl.font_manager._rebuild()


df = pd.read_csv("vgames2.csv") 
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]


df_sales = df.iloc[:,5:9]
df_sales = df_sales.replace({'K': '/1e3', 'M': ''}, regex = True).applymap(pd.eval).astype(float) # Sales column 의 K, M(단위) 에 대한 전처리.


drop_columns = ['NA_Sales', 'EU_Sales', 'JP_Sales','Other_Sales'] # 기존 df 에서 4개의 Sales column 과 "Name" column 도 같이 drop. Name 수가 너무 많기 때문에 유의미한 결과 도출이 어렵다고 판단.
df = df.drop(columns = drop_columns)
df = pd.concat([df, df_sales], axis = 1)


df['Global_Sales'] = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales'] # Global Sales 컬럼 추가.


# 결측치에 대한 전처리--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


# sns.heatmap(df.isnull())
# display(df[df['Year'].isnull()].head(10)) # 271 개의 데이터 N/A 아니면 NaN
# display(df[df['Year'].isnull()].shape)
# display(df[df['Publisher'].isnull()].head(10)) # 58 개의 데이터 N/A 아니면 NaN
# display(df[df['Publisher'].isnull()].shape)

# df['Publisher'].nunique() : 573개
# df['Genre'].nunique() : 12개
# df['Platform'].nunique() : 31개
# df['Name'].nunique() : 11,238개


df = df.dropna(axis = 0) # 결측치들은 NaN 혹은 N/A 포함하는 행을 모두 drop.
df['Year'] = df['Year'].astype(int) # Float -> Integer
df = df[df['Year'] > 100] # 100 보다 작은 연도 제거. 


# df.isnull().sum() # 결측치가 모두 사라진 것 확인 가능.


# 데이터 시각화(상관분석)---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


df.corr()
corr_df = df.iloc[:, 5:] # Sales 컬럼만 비교.


f, ax = plt.subplots(figsize=(7, 7))
sns.heatmap(corr_df.corr(), annot=True, linewidths=0.1, linecolor="gray", fmt= '.1f', ax=ax, cmap = "YlGn") # Correlation 에 대한 Heatmap.
plt.xticks(rotation=45)
plt.yticks(rotation=45)
ax.set_title("지역에 따른 매출액의 상관관계", pad=25, fontsize=21)
plt.tight_layout()
plt.show()


# 데이터 시각화(1. 지역에 따라서 선호하는 게임 장르가 다를까?)----------------------------------------------------------------------------------------------------------------------------------------------------


# 북미 Sales vs. 장르
sales_by_genre = df.groupby(['Genre']).sum()['NA_Sales'].sort_values(ascending = False)
plt.figure(figsize=(10,6))
sns.barplot(y = sales_by_genre.index, x = sales_by_genre.values, palette = "summer")
plt.title('북미 장르에 따른 매출액')
plt.xlabel('북미 매출액(단위: M(millions))')
plt.ylabel('장르')
plt.show()


# 유럽 Sales vs. 장르
sales_by_genre = df.groupby(['Genre']).sum()['EU_Sales'].sort_values(ascending = False)
plt.figure(figsize=(10,6))
sns.barplot(y = sales_by_genre.index, x = sales_by_genre.values, palette = "summer")
plt.title('유럽 장르에 따른 매출액')
plt.xlabel('유럽 매출액(단위: M(millions))')
plt.ylabel('장르')
plt.show()


# 일본 Sales vs. 장르
sales_by_genre = df.groupby(['Genre']).sum()['JP_Sales'].sort_values(ascending = False)
plt.figure(figsize=(10,6))
sns.barplot(y = sales_by_genre.index, x = sales_by_genre.values, palette = "summer")
plt.title('일본 장르에 따른 매출액')
plt.xlabel('일본 매출액(단위: M(millions))')
plt.ylabel('장르')
plt.show()


# 타지역 Sales vs. 장르
sales_by_genre = df.groupby(['Genre']).sum()['Other_Sales'].sort_values(ascending = False)
plt.figure(figsize=(10,6))
sns.barplot(y = sales_by_genre.index, x = sales_by_genre.values, palette = "summer")
plt.title('타지역 장르에 따른 매출액')
plt.xlabel('타지역 매출액(단위: M(millions))')
plt.ylabel('장르')
plt.show()


# 글로벌 Sales vs. 장르
sales_by_genre = df.groupby(['Genre']).sum()['Global_Sales'].sort_values(ascending = False)
plt.figure(figsize=(10,6))
sns.barplot(y = sales_by_genre.index, x = sales_by_genre.values, palette = "summer")
plt.title('전세계 장르에 따른 매출액')
plt.xlabel('전세계 매출액(단위: M(millions))')
plt.ylabel('장르')
plt.show()



# 데이터 시각화(2. 연도별 게임의 트렌드가 있을까?)----------------------------------------------------------------------------------------------------------------------------------------------------

  # 매출액과 연도 그리고 플랫폼에 대한 그래프를 Heatmap 을 이용하여 시각화.
platform_year_table = pd.pivot_table(df[df['Year'] <= 2020.0], values = ['Global_Sales'], index = ['Year'], columns = ['Platform'], aggfunc = 'sum')

plt.figure(figsize=(12,8))
sns.heatmap(platform_year_table['Global_Sales'], annot = True, annot_kws = {"size": 10}, fmt = 'g', cmap = "YlGn")
plt.xticks(rotation = 70)
plt.title('연도별 게임 플래폼 기준 매출액')
plt.xlabel('플래폼')
plt.ylabel('연도')
plt.show()



# 매출액과 연도 그리고 장르에 대한 그래프를 Heatmap 을 이용하여 시각화.
genre_year = pd.pivot_table(df[df['Year'] <= 2020.0], values = ['Global_Sales'], index = ['Year'], columns = ['Genre'], aggfunc = 'sum')

plt.figure(figsize=(12,8))
sns.heatmap(genre_year['Global_Sales'], annot = True, annot_kws = {"size": 10}, fmt = 'g', cmap = "YlGn")
plt.xticks(rotation = 70)
plt.title('연도별 장르에 따른 매출액(단위: M(millions))')
plt.xlabel('장르')
plt.ylabel('연도')
plt.show()


# 발매한 게임 수 에 따른 TOP 10 제작사 에 대한 시각화.
games_by_publisher = df.groupby(['Publisher']).count()['Name'].sort_values(ascending=False)[:10]
plt.figure(figsize=(10,6))
sns.barplot(y = games_by_publisher.index, x = games_by_publisher.values, palette = "summer")
plt.title('제작사에 따른 게임 발매 수량')
plt.xlabel('게임 발매 수량')
plt.ylabel('제작사')
plt.show()


# 누적 매출액 기준으로 TOP 10 제작사에 대한 시각화.
sales_by_publisher = df.groupby(['Publisher']).sum()['Global_Sales'].sort_values(ascending = False)[:10]
plt.figure(figsize=(10,6))
sns.barplot(y = sales_by_publisher.index, x = sales_by_publisher.values, palette = "summer")
plt.title('제작사에 따른 누적 매출액')
plt.xlabel('매출액(단위 : M(millions)')
plt.ylabel('제작사')
plt.show()


# 제작사별 발매한 게임 1개 기준 생성되는 매출액(Return per game 값)에 대한 시각화.
num_games = df.groupby(['Publisher']).count()['Name']
sales_rev = df.groupby(['Publisher']).sum()['Global_Sales']
revenue_per_game = (sales_rev / num_games).sort_values(ascending = False)[:10]
plt.figure(figsize=(10,6))
sns.barplot(y = revenue_per_game.index, x = revenue_per_game.values, palette = "summer")
plt.title('제작사에 따른 출시한 게임 수량 대비 매출액')
plt.xlabel('출시 게임 개당 매출액 단위: M(millions)')
plt.ylabel('제작사')
plt.show()


# 연도에 따른 제작사 별 매출액에 대한 시각화.
top_10_pub = df.groupby(['Publisher']).sum()['Global_Sales'].sort_values(ascending = False)[:10]
publisher_year_table = pd.pivot_table(df[df['Publisher'].isin(top_10_pub.index)][df['Year'] <= 2020.0], values = ['Global_Sales'], index = ['Year'], columns = ['Publisher'], aggfunc = 'sum')

plt.figure(figsize=(12,8))
sns.heatmap(publisher_year_table['Global_Sales'], annot = True, annot_kws = {"size": 10}, fmt = 'g', cmap = "YlGn")
plt.xticks(rotation = 70)
plt.title('연도에 따른 제작사 별 매출액 (단위: M(millions))')
plt.xlabel('제작사')
plt.ylabel('연도')
plt.show()


# 매출액에 따른 제작사와 장르에 대한 시각화.
publisher_genre_table = pd.pivot_table(df[df['Publisher'].isin(top_10_pub.index)][df['Year'] <= 2020], values = ['Global_Sales'], index = ['Genre'], columns = ['Publisher'], aggfunc = 'sum')

plt.figure(figsize=(12,6))
sns.heatmap(publisher_genre_table['Global_Sales'], annot = True, annot_kws = {"size": 10}, fmt = 'g', cmap = "YlGn")
plt.xticks(rotation = 70)
plt.title('매출액에 따른 제작사와 장르(단위: M(millions))')
plt.xlabel('제작사')
plt.ylabel('장르')
plt.show()
